# **Text mining: SENTIMENT ANALYSIS**

## 🎓 Master’s Program in Data Science & Advanced Analytics  
**Nova IMS** | June 2025  
**Course:** Text Mining

## 👥 Team **Group 34**  
- **[Philippe Dutranoit]** | [20240518]  
- **[Diogo Duarte]** | [20240525]  
- **[Rui luz]** | [20211628]  
- **[Rodrigo Sardinha]** | [20211627]  

## 📊 Goal of the notebook


In this notebook we finalize our approach using the model and the approach that gave use the best results. we finalize the project by making the prediction on the test set for the model evaluation

# Imports 

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df_test = pd.read_csv('../Data/test.csv')
